In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
import re

import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
num_features = 64
num_labels = 7
batch_size = 200
epochs = 100
width, height = 48, 48

In [ ]:
data_train = pd.read_csv("sampled_emotions.csv", sep=",",)

data_train = data_train.values
Y_train = data_train[:,0]
X_training = data_train[:,1:]

In [ ]:
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [ ]:
X_train = []

for i in range(len(X_training)):
    cc = X_training[i][0]
    cc = re.sub("\s+", ",", cc.strip())
    cc = np.fromstring(cc, dtype=int, sep=',')
    #print(cc.shape)
    ccb = preprocess_input(cc)
    a = np.reshape(ccb,(48,48,1))
    X_train.append(a)

X_train = np.array(X_train)
#img_shape_full = np.array(X_train)
#X_Test

In [ ]:
data_test = pd.read_csv("sampled_emotions_test_clean.csv", sep=",",)

data_test = data_test.values
Y_Test = data_test[:,1]
X_Testing = data_test[:,2:]

#X_Testing

In [ ]:
def preprocess_input(y, v2=True):
    y = y.astype('float32')
    y = y / 255.0
    if v2:
        y = y - 0.5
        y = y * 2.0
    return y

In [ ]:

X_Test = []

for i in range(len(X_Testing)):
    bb = X_Testing[i][0]
    bb = re.sub("\s+", ",", bb.strip())
    bb = np.fromstring(bb, dtype=int, sep=',')
    #print(cc.shape)bb
    cb = preprocess_input(bb)
    b = np.reshape(cb,(48,48,1))
    X_Test.append(b)

X_Test = np.array(X_Test)
#img_shape_full = np.array(X_train)
#X_Test

In [ ]:
p = X_train.shape[0]
#g = int(round((0.85)*p))
#h = int(round((0.15)*p))
    #v = int((0.15)*p)
    
x_train = X_train[:]
y_train = Y_train[:]
#x_valid = X_train[g:]
#y_valid = Y_train[g+1:]

In [ ]:
#x_train.shape

In [ ]:
y_train_hot= np.zeros((y_train.shape[0], len(np.unique(y_train))))
y_train_hot.shape

In [ ]:
#y_valid_hot= np.zeros((y_valid.shape[0], len(np.unique(y_valid))))
#y_valid_hot.shape

In [ ]:
y_test_hot= np.zeros((Y_Test.shape[0], len(np.unique(Y_Test))))
y_test_hot.shape

In [ ]:
y_train_hot = y_train_hot.astype(int)
#y_train_hot
#y_valid_hot = y_valid_hot.astype(int)
#y_valid_hot
y_test_hot = y_test_hot.astype(int)
#y_test_hot

for i in range(0, y_train.shape[0]):
    y_train_hot[i][y_train[i]]= 1

In [ ]:
for i in range(0, Y_Test.shape[0]):
    y_test_hot[i][Y_Test[i]]= 1

In [ ]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
tensorboard = TensorBoard(log_dir='./logs')

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)

In [ ]:
history = model.fit(np.array(X_train), np.array(y_train_hot),
          batch_size=batch_size,
          epochs= epochs,
          verbose=1,
          validation_data=(np.array(X_Test), np.array(y_test_hot)),
          shuffle=True,
          callbacks=[ tensorboard, checkpointer,lr_reducer])